# Activo la libreria de drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Verificacion tecnica del sistema.



In [4]:
from platform import python_version
import tensorflow as tf

print("Version de python: " + python_version())
print("Version de tensorflow: " + tf.__version__)


Version de python: 3.7.10
Version de tensorflow: 2.5.0


# Importamos librerias.

In [5]:
# import the libraries as shown below
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import keras.backend as K

In [6]:
# re-size all the images to this

train_path = '/content/drive/MyDrive/AgricultureVisión/Data/dataset-estadistica/train/rgb'
valid_path = '/content/drive/MyDrive/AgricultureVisión/Data/dataset-estadistica/test/rgb'

In [7]:
K.clear_session()

def build_model(num_classes, img_size=512):
    input = tf.keras.layers.Input(shape=(img_size, img_size, 3))
    model = tf.keras.applications.NASNetLarge(include_top=False, input_tensor=input, weights="imagenet")

    # Freeze the pretrained weights
    # model.trainable = True
    for layer in model.layers[:-1]:
      layer.trainable = False

    # Rebuild top
    x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = tf.keras.layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = tf.keras.layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    output = tf.keras.layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(input, output, name="NASNetLarge")
    # model.summary()
    model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["categorical_accuracy"])
    return model

In [8]:
# useful for getting number of output classes
folders = glob(train_path + '/*')
num_imagenes = glob(train_path + '/*/*')

In [9]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
batch_size_ = 32

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (512, 512),
                                                 batch_size = batch_size_,
                                                 class_mode = 'categorical')

Found 1080 images belonging to 9 classes.


In [11]:
test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (512, 512),
                                            batch_size = batch_size_,
                                            class_mode = 'categorical')

Found 450 images belonging to 9 classes.


In [12]:
num_classes = len(folders)
num_images = len(num_imagenes)

model = build_model(num_classes)

343613440/343610240 [==============================] - 2s 0us/step


In [ ]:
import time
import math

def entrenamiento_cpu():
  with tf.device('/cpu:0'):
    history = model.fit(
        training_set,
        validation_data=test_set,
        epochs=50,
        steps_per_epoch=math.ceil(num_images/batch_size_))
  return history

inicio = time.time()
history = entrenamiento_cpu()
fin = time.time()

Epoch 1/50
34/34 [==============================] - 1889s 55s/step - loss: 1.5365 - categorical_accuracy: 0.5093 - val_loss: 1.7637 - val_categorical_accuracy: 0.4689
Epoch 2/50
34/34 [==============================] - 1876s 56s/step - loss: 0.8418 - categorical_accuracy: 0.7074 - val_loss: 1.6678 - val_categorical_accuracy: 0.4933
Epoch 3/50
34/34 [==============================] - 1876s 56s/step - loss: 0.6077 - categorical_accuracy: 0.7880 - val_loss: 1.5521 - val_categorical_accuracy: 0.5489
Epoch 4/50
34/34 [==============================] - 1829s 54s/step - loss: 0.5171 - categorical_accuracy: 0.8139 - val_loss: 1.4606 - val_categorical_accuracy: 0.5622
Epoch 5/50
34/34 [==============================] - 1871s 55s/step - loss: 0.4436 - categorical_accuracy: 0.8565 - val_loss: 1.3951 - val_categorical_accuracy: 0.5489
Epoch 6/50
34/34 [==============================] - 1838s 54s/step - loss: 0.3871 - categorical_accuracy: 0.8694 - val_loss: 1.3605 - val_categorical_accuracy: 0.591

In [ ]:
print("Tiempo de entrenamiento cpu flow_from_directory " + str((fin-inicio)/60) + " minutos")

In [ ]:
def plotTraining(hist, epochs, typeData):
    
    if typeData=="loss":
        plt.figure(1,figsize=(10,5))
        yc=hist.history['loss']
        xc=range(epochs)
        plt.ylabel('Costo', fontsize=24)
        plt.plot(xc,yc,'-r',label='Costo entrenamiento')
    if typeData=="categorical_accuracy":
        plt.figure(2,figsize=(10,5))
        yc=hist.history['categorical_accuracy']
        for i in range(0, len(yc)):
            yc[i]=100*yc[i]
        xc=range(epochs)
        plt.ylabel('Accuracy (%)', fontsize=24)
        plt.plot(xc,yc,'-r',label='Accuracy entrenamiento')
    if typeData=="val_loss":
        plt.figure(1,figsize=(10,5))
        yc=hist.history['val_loss']
        xc=range(epochs)
        plt.ylabel('Costo', fontsize=24)
        plt.plot(xc,yc,'--b',label='Validación costo')
    if typeData=="val_categorical_accuracy":
        plt.figure(2,figsize=(10,5))
        yc=hist.history['val_categorical_accuracy']
        for i in range(0, len(yc)):
            yc[i]=100*yc[i]
        xc=range(epochs)
        plt.ylabel('Accuracy (%)', fontsize=24)
        plt.plot(xc,yc,'--b',label='Validación accuracy')
        

    plt.rc('xtick',labelsize=24)
    plt.rc('ytick',labelsize=24)
    plt.rc('legend', fontsize=18) 
    plt.legend()
    plt.xlabel('Numero de epocas',fontsize=24)
    plt.grid(True)

In [ ]:
epochs = 50
plotTraining(history,epochs,"loss")
plotTraining(history,epochs,"categorical_accuracy")
plotTraining(history,epochs,"val_loss")
plotTraining(history,epochs,"val_categorical_accuracy")

In [ ]:
from tensorflow.keras.models import load_model

model.save('/content/drive/MyDrive/AgricultureVisión/notebooks/mineria/Model/nasnet_50epo_from_directory_cpu.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
K.clear_session()

def build_model(num_classes, img_size=512):
    input = tf.keras.layers.Input(shape=(img_size, img_size, 3))
    model = tf.keras.applications.NASNetLarge(include_top=False, input_tensor=input, weights="imagenet")

    # Freeze the pretrained weights
    # model.trainable = True
    for layer in model.layers[:-1]:
      layer.trainable = False

    # Rebuild top
    x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = tf.keras.layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = tf.keras.layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    output = tf.keras.layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(input, output, name="NASNetLarge")
    # model.summary()
    model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["categorical_accuracy"])
    return model

In [ ]:
import time
import math

def entrenamiento_cpu():
  with tf.device('/cpu:0'):
    history = model.fit(
        training_set,
        validation_data=test_set,
        epochs=10,
        steps_per_epoch=math.ceil(num_images/batch_size_))
  return history

inicio = time.time()
history = entrenamiento_cpu()
fin = time.time()

In [ ]:
print("Tiempo de entrenamiento cpu flow_from_directory " + str((fin-inicio)/60) + " minutos")

In [ ]:
epochs = 10
plotTraining(history,epochs,"loss")
plotTraining(history,epochs,"categorical_accuracy")
plotTraining(history,epochs,"val_loss")
plotTraining(history,epochs,"val_categorical_accuracy")

In [ ]:
from tensorflow.keras.models import load_model

model.save('/content/drive/MyDrive/AgricultureVisión/notebooks/mineria/Model/nasnet_10epo_from_directory_cpu.h5')